In [39]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [40]:
hotel = pd.read_csv('../../hoteles/hoteles-mad-dummies.csv')
restaurantes = pd.read_csv('../../restaurantes/restaurantes-mad-dummies.csv')
planes = pd.read_csv('../../turismo/turismo-mad-dummies.csv')
geo = gpd.read_file('../../proyecto/mapa-madrid/limites-barrios-madrid.geojson')

In [41]:
planes.insert(6, 'punto', 0)

In [42]:
planes['punto'] = gpd.GeoSeries.from_xy(planes.long, planes.lat)

In [43]:
barrio = []
            
for p in planes.punto:
    
    encontrado = False
    
    for i, poly in enumerate(geo.geometry):
        if poly.contains(p):
            barrio.append(geo.name[i])
            encontrado = True
            break
            
    if not encontrado:
        barrio.append("Barrio no encontrado")


In [44]:
planes.insert(4, 'barrio', barrio)

In [45]:
hotel.insert(7, 'punto', 0)

In [46]:
hotel['punto'] = gpd.GeoSeries.from_xy(hotel.long, hotel.lat)

In [47]:
barrio1 = []
            
for p in hotel.punto:
    
    encontrado = False
    
    for i, poly in enumerate(geo.geometry):
        if poly.contains(p):
            barrio1.append(geo.name[i])
            encontrado = True
            break
            
    if not encontrado:
        barrio1.append("Barrio no encontrado")

In [48]:
hotel.insert(5, 'barrio', barrio1)

In [49]:
restaurantes.insert(7, 'punto', 0)

In [50]:
restaurantes['punto'] = gpd.GeoSeries.from_xy(restaurantes.long, restaurantes.lat)

In [51]:
barrio2 = []
            
for p in restaurantes.punto:
    
    encontrado = False
    
    for i, poly in enumerate(geo.geometry):
        if poly.contains(p):
            barrio2.append(geo.name[i])
            encontrado = True
            break
            
    if not encontrado:
        barrio2.append("Barrio no encontrado")

In [52]:
restaurantes.insert(3, 'barrio', barrio2)

In [69]:
hotel.to_csv('../csv/hoteles-def.csv')
restaurantes.to_csv('../csv/restaurantes-def.csv')
planes.to_csv('../csv/planes-def.csv')

In [24]:
planes = planes[['nombre','puntuacion','barrio','lat', 'long']]
hotel = hotel[['nombre','puntuacion','barrio','lat', 'long']]
restaurantes = restaurantes[['nombre','puntuacion','barrio','lat', 'long']]

In [25]:
restaurantes.insert(1,'tipo', 'Restaurante')
hotel.insert(1, 'tipo', 'Hotel')
planes.insert(1, 'tipo', 'Ocio')

In [36]:
restaurantes.head()

,nombre,tipo,puntuacion,barrio,lat,long
0,Spicy Soul Hot Pot,Restaurante,"5,0",Castellana,40.435127,-3.684034
1,Los Montes de Galicia,Restaurante,"5,0",Guindalera,40.434746,-3.668260
2,Empanadas Malvón,Restaurante,"5,0",Trafalgar,40.431938,-3.698437
3,Pastamore,Restaurante,"5,0",Jerónimos,40.420696,-3.685145
4,Amazonia Brunch And Cocktail,Restaurante,"5,0",Universidad,40.422791,-3.703948


In [26]:
mapa_madrid = pd.concat([restaurantes, hotel, planes], axis = 0)

In [27]:
planes.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1543 entries, 0 to 1542
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   nombre      1543 non-null   object 
 1   tipo        1543 non-null   object 
 2   puntuacion  1543 non-null   object 
 3   barrio      1543 non-null   object 
 4   lat         1543 non-null   float64
 5   long        1543 non-null   float64
dtypes: float64(2), object(4)
memory usage: 452.2 KB


In [28]:
for c in planes.select_dtypes('float'):
    
    planes[c]=pd.to_numeric(planes[c], downcast='float')


In [29]:
hotel.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 716 entries, 0 to 715
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   nombre      716 non-null    object 
 1   tipo        716 non-null    object 
 2   puntuacion  716 non-null    object 
 3   barrio      716 non-null    object 
 4   lat         716 non-null    float64
 5   long        716 non-null    float64
dtypes: float64(2), object(4)
memory usage: 205.9 KB


In [30]:
for c in hotel.select_dtypes('float'):
    
    hotel[c]=pd.to_numeric(hotel[c], downcast='float')


In [31]:
restaurantes.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6308 entries, 0 to 6307
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   nombre      6308 non-null   object 
 1   tipo        6308 non-null   object 
 2   puntuacion  6308 non-null   object 
 3   barrio      6308 non-null   object 
 4   lat         6308 non-null   float64
 5   long        6308 non-null   float64
dtypes: float64(2), object(4)
memory usage: 1.8 MB


In [32]:
for c in restaurantes.select_dtypes('float'):
    
    restaurantes[c]=pd.to_numeric(restaurantes[c], downcast='float')
    

In [33]:
mapa = folium.Map(location=[40.4167278, -3.7033387], zoom_start=15, tiles = 'Stamen Terrain')

In [34]:
for i in range(0, len(restaurantes)):

    popup_txt = folium.Popup(restaurantes.nombre[i], restaurantes.tipo[i])

    folium.Marker([float(restaurantes.lat[i]), float(restaurantes.long[i])], 
                        popup=popup_txt,
                        icon = folium.Icon(color = 'pink')).add_to(mapa)

for i in range(0, len(hotel)):

    popup_txt = folium.Popup(hotel.nombre[i], hotel.tipo[i])

    folium.Marker([float(hotel.lat[i]), float(hotel.long[i])], 
                        popup=popup_txt,
                        icon = folium.Icon(color = 'green')).add_to(mapa)

for i in range(0, len(planes)):

    popup_txt = folium.Popup(planes.nombre[i], planes.tipo[i])

    folium.Marker([float(planes.lat[i]), float(planes.long[i])], 
                        popup=popup_txt,
                        icon = folium.Icon(color = 'green')).add_to(mapa)


In [35]:
mapa.save('../mapa-madrid/mapa.html')

In [37]:
rest = restaurantes.sample(n=10)
hoteles = hotel.sample(n=10)
ocio = planes.sample(n=10)


In [62]:
planes.puntuacion.unique()

array(['4.5', '4.0', '5.0', '3.5', '2.5', '3.0', '2.0', '1.0',
       'Esta actividad todavía no tiene puntuación, ¡se el primero!',
       '1.5', '-1.0'], dtype=object)

In [63]:
planes.puntuacion = planes.puntuacion.replace('Esta actividad todavía no tiene puntuación, ¡se el primero!', '0')

In [66]:
planes.puntuacion = planes.puntuacion.astype('float')

In [68]:
ocio.to_csv('planes_mapa.csv', index = False)
hoteles.to_csv('hotel_mapa.csv', index = False)
rest.to_csv('restaurantes_mapa.csv', index = False)
mapa_madrid.to_csv('mapa_madrid.csv', index = False)

In [71]:
hotel.head()

,descripcion,nombre,precio,puntuacion,ubicacion,barrio,lat,long,punto,Aparcamiento privado de pago cerca,Internet de alta velocidad gratuito (wifi),Jacuzzi,Bar/salón,Actividades para niños (acondicionado para familias / niños),Servicio de taxi,Centro de negocios con acceso a Internet,Salas de reuniones,Albornoces,Aire acondicionado,Mesa de escritorio,Servicio de limpieza,Servicio de habitaciones,Cafetera/tetera,Televisor de pantalla plana,Bañera/ducha,Aparcamiento público de pago cerca,Gimnasio / Sala de entrenamiento,Envoltura corporal,Cortinas opacas,Minibar,TV por cable o satélite,Camas extragrandes,Ducha a ras de suelo,Piscina,Transporte desde/al aeropuerto,Terraza en el ático,Vigilancia 24 horas,Balcón privado,Frigorífico,Aparcamiento privado de pago dentro del recinto,Alquiler de coches,Instalaciones para conferencias,Habitación hipoalergénica,Suite nupcial,Espacio para guardar el equipaje,Conserje,Recepción 24 horas,Lavado en seco,Caja fuerte,Secador de pelo,Aparcamiento en la calle,Cafetería,Teléfono,Artículos de aseo gratuitos de cortesía,Guardería,Sofá cama,Bañera de hidromasaje,Punto de recarga de vehículos eléctricos,Alquiler de bicicletas,Pub/DJ,Se admiten mascotas,Servicio de aparcacoches,Desayuno gratuito,Desayuno disponible,Registro de entrada 24 horas,Tronas disponibles,Carritos de bebé,Canales de televisión infantiles,Piscina en la azotea,Zona para tomar el sol,Terraza-solario,Sofá,Aparcamiento,Masaje,Equipamiento para sala VIP,Películas bajo demanda,Cocina americana,Microondas,Consola de videojuegos,Transporte gratuito desde/al aeropuerto,Animación nocturna,Garaje,Wifi,Autoservicio de lavandería,Internet gratuito,Hotel de no fumadores,Armario,Perchero para la ropa,Servicio de lavandería,Chimenea,Billares,Dardos,Registro de entrada y salida exprés,Piscina de poca profundidad,Piscina exterior,Máquina expendedora,Estancia gratuita para niños,Salón de belleza,Solario,Sala de banquetes,Bicicletas disponibles,Wifi pública,Bidé,Café instantáneo de cortesía,Desayuno bufé,Servicio despertador/reloj con alarma,Wifi de pago,Servicio de planchado,Aparcamiento público de pago dentro del recinto,Cocina compartida,Clases de yoga,Sala de juegos,Sauna,Hervidor eléctrico,Karaoke,Baños privados,Secadora,Plancha,Balneario,Caja fuerte para portátiles,Piscina infinita,Restaurante,Casino y juegos de azar,Comedor,Prensa,Visitas guiadas a pie,Horno,Bar junto a la piscina,Zona de asientos,Comedor independiente,Agua embotellada,Aparcamiento gratuito,Lavadora,Servicio de traslado en autobús,Unnamed: 135,Personal de animación,Patio,Vestuarios en gimnasio o balneario,Tratamientos faciales,Zona infantil,Internet,Purificador de aire,Aparcamiento privado gratuito cerca,Servicio de lustrado de zapatos,Servicio de desayuno en la habitación,Salón compartido/sala de televisión,Piscina con vistas,Solo para adultos,Kit de primeros auxilios,Baño adicional,Juegos de mesa/puzles,Registro privado de entrada y salida,Té de cortesía,Libros,DVD y música para niños,Cambio de divisas,Zona infantil de juegos cubierta,Puerto para iPod,Piscina climatizada,Rutas en bicicleta,Baño compartido,Bar de tapas,Paraguas,Servicios gratuitos de traslado en autobús o taxi,Senderismo,Pista de tenis,Bar en la azotea,Lavavajillas,Masaje de cuerpo entero,Campo de golf,Tiendas,Suelo de baldosas o mármol,Fotocopiadora/fax en el centro de negocios,Hammam (baño turco),Aparcamiento vigilado,Mobiliario exterior,Aparcamiento público gratuito cerca,Instalaciones para hacer barbacoas,Sala de estar independiente,Utensilios de cocina,Piscina vallada,Internet de pago,Radio,Habitaciones de no fumadores,Suites,Habitaciones insonorizadas,Vistas a la ciudad,Habitaciones para familias,Habitaciones interconectadas disponibles,Vistas a sitios de interés turístico,Habitaciones para fumadores disponibles,Vistas a la montaña,Unnamed: 193,Vistas a la piscina
0,El Heritage Madrid Hotel cuenta con 46 habitac...,Relais & Chateaux Heritage Madrid Hotel,354,5.0,"Calle de Diego de León, 43, 28006, Madr